In [14]:
import pandas as pd
import talib
import random

In [2]:
data_df = pd.read_csv("../data/LTCUSDT-30m-data.csv")

In [3]:
data_df

,timestamp,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
0,2017-12-13 03:30:00.000,272.00,288.00,272.00,276.00,130.86075,1513137599999,3.623134e+04,65,11.87562,3.319173e+03,265049.598083
1,2017-12-13 04:00:00.000,276.00,284.10,269.99,270.00,350.86428,1513139399999,9.616079e+04,77,44.46382,1.216524e+04,264975.004646
2,2017-12-13 04:30:00.000,270.00,288.00,270.00,279.00,255.61702,1513141199999,7.056886e+04,88,92.45727,2.570841e+04,264696.549732
3,2017-12-13 05:00:00.000,279.00,320.00,279.00,287.00,220.08266,1513142999999,6.384420e+04,82,148.75093,4.300407e+04,262766.913647
4,2017-12-13 05:30:00.000,287.00,330.00,282.00,302.00,163.68408,1513144799999,4.916537e+04,87,89.86945,2.767100e+04,261070.341130
...,...,...,...,...,...,...,...,...,...,...,...,...
50958,2020-11-13 07:00:00,62.55,62.55,61.58,61.61,26435.73662,1605252599999,1.643066e+06,3487,9861.91699,6.133267e+05,0.000000
50959,2020-11-13 07:30:00,61.64,62.40,61.25,62.30,45162.19962,1605254399999,2.788383e+06,5393,22479.94228,1.388100e+06,0.000000
50960,2020-11-13 08:00:00,62.27,62.56,61.54,61.92,33160.95155,1605256199999,2.055975e+06,4537,16447.27198,1.019831e+06,0.000000
50961,2020-11-13 08:30:00,61.92,63.67,61.73,63.54,55368.46795,1605257999999,3.480437e+06,7976,34112.11884,2.146223e+06,0.000000


In [17]:
params = {
    'RSI_BULL': 10,
    'RSI_BULL_TF': 19,
    'RSI_BULL_HIGH': 81.93600697495238,
    'RSI_BULL_LOW' : 45.516632226052714,
    'RSI_BEAR': 15,
    'RSI_BEAR_TF': 16,
    'RSI_BEAR_HIGH': 45.61985405602511,
    'RSI_BEAR_LOW' : 20.925647327774335,
    'SMA_FAST': 138,
    'SML_SLOW': 67,
    'SMA_TF': 6,
    'ADX': 3,
    'ADX_TF': 9,
    'ADX_HIGH': 65.80525318969225,
    'ADX_LOW': 45.6513373850286,
    'BULL_MOD_HIGH': 5,
    'BULL_MOD_LOW' : -5,
    'BEAR_MOD_HIGH': 15,
    'BEAR_MOD_LOW' : -5,
    'STOP_LOSS': 14, # %,
    'TAKE_PROFIT': 28 # %
}
params2 = {'RSI_BULL': 14, 'RSI_BULL_TF': 6, 'RSI_BULL_HIGH': 70.66602096693327, 'RSI_BULL_LOW': 47.72774022537839, 'RSI_BEAR': 19, 'RSI_BEAR_TF': 20, 'RSI_BEAR_HIGH': 47.831780740039356, 'RSI_BEAR_LOW': 19.675569028578884, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 9, 'ADX': 15, 'ADX_TF': 5, 'ADX_HIGH': 62.76900764282739, 'ADX_LOW': 30.616489167985236, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 12.475810949883751, 'TAKE_PROFIT': 10.09271064841754}

In [5]:
def run_strategy(data, params):
    # calculating
    rsi_bull = talib.RSI(data['close'], timeperiod=params['RSI_BULL'])
    rsi_bear = talib.RSI(data['close'], timeperiod=params['RSI_BEAR'])
    sma_slow = talib.SMA(data['close'], timeperiod=params['SML_SLOW'])
    sma_fast = talib.SMA(data['close'], timeperiod=params['SMA_FAST'])
    adx = talib.ADX(high=data['high'],low=data['low'], close=data['close'], timeperiod=params['ADX'])


    order_history = []
    entry_price = 0
    exit_price = 0
    dd = 0
    ru = 0

    sma_timeframe = params['SMA_TF']
    adx_timeframe = params['ADX_TF']
    rsi_bull_timeframe = params['RSI_BULL_TF']
    rsi_bear_timeframe = params['RSI_BEAR_TF']
    rsi_bull_high = params['RSI_BULL_HIGH']
    rsi_bull_low = params['RSI_BULL_LOW']
    rsi_bear_high = params['RSI_BEAR_HIGH']
    rsi_bear_low = params['RSI_BEAR_LOW']
    adx_high = params['ADX_HIGH']
    adx_low = params['ADX_LOW']
    bull_mod_high = params['BULL_MOD_HIGH']
    bull_mod_low = params['BULL_MOD_LOW']
    bear_mod_high = params['BEAR_MOD_HIGH']
    bear_mod_low = params['BEAR_MOD_LOW']
    stop_loss = params['STOP_LOSS']  # %
    take_profit = params['TAKE_PROFIT']  # %

    sma_slow_count = 0
    sma_fast_count = 0
    adx_count = 0
    rsi_bull_count = 0
    rsi_bear_count = 0

    in_position = False
    in_fake_position = False  # position if dont have stop loss/take profit
    sma_fast_value = 0
    sma_slow_value = 0
    adx_value = 0


    for index, row in data.iterrows():
        price = row['close']
        # SMA slow
        if sma_slow_count >= sma_timeframe:
            sma_slow_value = sma_slow[index]
            sma_slow_count = 0
        else:
            sma_slow_count += 1

        # SMA fast
        if sma_fast_count >= sma_timeframe:
            sma_fast_value = sma_fast[index]
            sma_fast_count = 0
        else:
            sma_fast_count += 1

        #ADX
        if adx_count >= adx_timeframe:
            adx_value = adx[index]
            adx_count = 0
        else:
            adx_count += 1

        # RSI bull
        rsi_bull_value = rsi_bull[index]

        # RSI bear
        rsi_bear_value = rsi_bear[index]

        is_bull_market = sma_fast_value > sma_slow_value
        is_bear_market = sma_fast_value < sma_slow_value

        # dd, ru
        if in_position:
            dd = min(dd, price)
            ru = max(ru, price)
            # stop/take profit
            if (1 - price / entry_price) > stop_loss / 100 or (price/entry_price - 1) > take_profit / 100:
                # print("Sell at: ", price)
                # print("------------")
                exit_price = price
                # save history
                order_history.append(
                    {"entry": entry_price, "exit": exit_price, "profit": 100 * (exit_price / entry_price - 1), 
                     "draw_down": dd, "run_up": ru})
                in_position = False
        else:
            dd = 0
            ru = 0

        if is_bull_market:
            if adx_value > adx_high:
                rsi_bull_high_adx = rsi_bull_high + bull_mod_high
            elif adx_value < adx_low:
                rsi_bull_low_adx = rsi_bull_low + bull_mod_low
            if not in_position and rsi_bull_value >= rsi_bull_high:
                # print("------------")
                # print("Buy at: ", price)
                in_position = True
                entry_price = price
                dd = ru = price
            elif in_position and rsi_bull_value <= rsi_bull_low:
                # print("Sell at: ", price)
                # print("------------")
                exit_price = price
                # save history
                order_history.append(
                    {"entry": entry_price, "exit": exit_price, "profit": 100 * (exit_price / entry_price - 1), 
                     "draw_down": dd, "run_up": ru})
                in_position = False

        elif is_bear_market:
            if adx_value > adx_high:
                rsi_bear_high_adx = rsi_bear_high + bear_mod_high
            elif adx_value < adx_low:
                rsi_bear_low_adx = rsi_bear_low + bear_mod_low
            if not in_position and rsi_bear_value >= rsi_bear_high:
                # print("------------")
                # print("Buy at: ", price)
                in_position = True
                entry_price = price
                dd = ru = price
            elif in_position and rsi_bear_value <= rsi_bear_low:
                # print("Sell at: ", price)
                # print("------------")
                exit_price = price
                order_history.append(
                    {"entry": entry_price, "exit": exit_price, "profit": 100 * (exit_price / entry_price - 1), 
                     "draw_down": dd, "run_up": ru})
                in_position = False
    return pd.DataFrame(order_history)

In [18]:
order_history_df = run_strategy(data_df, params2)

In [19]:
order_history_df

,entry,exit,profit,draw_down,run_up
0,297.00,330.00,11.111111,282.05,330.00
1,330.00,363.31,10.093939,330.00,363.31
2,363.31,315.71,-13.101759,315.71,363.61
3,346.45,289.00,-16.582479,289.00,349.55
4,329.32,302.62,-8.107616,300.00,336.49
...,...,...,...,...,...
310,60.24,57.77,-4.100266,57.77,60.24
311,56.19,55.75,-0.783057,55.75,56.19
312,55.43,53.96,-2.651994,53.51,56.54
313,55.43,61.38,10.734259,55.11,61.38


In [8]:
order_history_df.to_csv("LTCUSD-30m-results.csv")

In [9]:
def performance_report(order_df):
    total_profit = order_df['profit'].sum()
    max_dd = order_df['draw_down'].min()      
    max_ru = order_df['run_up'].max()
    num_win = order_df[order_df.profit >= 0].count().profit
    num_loss = order_df[order_df.profit < 0].count().profit
    total_win_profit = order_df[order_df.profit >= 0].sum().profit
    total_loss_profit = order_df[order_df.profit < 0].sum().profit
    ave_win = total_win_profit/num_win
    ave_loss = total_loss_profit/num_loss
    win_rate = num_win/(num_win+num_loss)
    return {'total_profit': total_profit, 'max_dd': max_dd, 'max_ru':max_ru, 
            'num_win':num_win,'num_loss':num_loss,
            'total_win_profit':total_win_profit, 
            'total_loss_profit':total_loss_profit, 
            'ave_win':ave_win, 'ave_loss':ave_loss,
            'win_rate':win_rate}

In [20]:
performance_report(order_history_df)

{'total_profit': 124.17595231450329,
 'max_dd': 22.95,
 'max_ru': 363.61,
 'num_win': 118,
 'num_loss': 197,
 'total_win_profit': 871.9838569821809,
 'total_loss_profit': -747.8079046676781,
 'ave_win': 7.389693703238821,
 'ave_loss': -3.7959792115110567,
 'win_rate': 0.3746031746031746}

In [16]:
# backtest
# Brute force
total_profit = 0
while True:
    params = {
        'RSI_BULL': random.randint(5, 20),
        'RSI_BULL_TF': random.randint(5, 20),
        'RSI_BULL_HIGH': random.uniform(60, 100),
        'RSI_BULL_LOW' : random.uniform(40, 60),
        'RSI_BEAR': random.randint(5, 20),
        'RSI_BEAR_TF': random.randint(5, 20),
        'RSI_BEAR_HIGH': random.uniform(30, 60),
        'RSI_BEAR_LOW' : random.uniform(5, 30),
        'SMA_FAST': 138,
        'SML_SLOW': 67,
        'SMA_TF': random.randint(5, 20),
        'ADX': random.randint(5, 20),
        'ADX_TF': random.randint(5, 20),
        'ADX_HIGH': random.uniform(60, 100),
        'ADX_LOW': random.uniform(5, 60),
        'BULL_MOD_HIGH': 5,
        'BULL_MOD_LOW' : -5,
        'BEAR_MOD_HIGH': 15,
        'BEAR_MOD_LOW' : -5,
        'STOP_LOSS': random.uniform(5, 15), # %,
        'TAKE_PROFIT': random.uniform(10, 30) # %
    }
    odf = run_strategy(data_df, params)
    pr  = performance_report(odf)
    if total_profit <= pr['total_profit']:
        total_profit = pr['total_profit']
        print("------------------------")
        print ("Best profit: ", total_profit)
        print ("Params: ", params)
        print("------------------------")

------------------------
Best profit:  28.37833694564992
Params:  {'RSI_BULL': 17, 'RSI_BULL_TF': 14, 'RSI_BULL_HIGH': 71.4306606454597, 'RSI_BULL_LOW': 58.13039826486809, 'RSI_BEAR': 17, 'RSI_BEAR_TF': 19, 'RSI_BEAR_HIGH': 44.62360196854659, 'RSI_BEAR_LOW': 20.556662811205236, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 20, 'ADX': 18, 'ADX_TF': 12, 'ADX_HIGH': 74.64628489004137, 'ADX_LOW': 10.528534710577773, 'BULL_MOD_HIGH': 5, 'BULL_MOD_LOW': -5, 'BEAR_MOD_HIGH': 15, 'BEAR_MOD_LOW': -5, 'STOP_LOSS': 5.135739605995169, 'TAKE_PROFIT': 18.753906262038633}
------------------------
------------------------
Best profit:  84.19699915931737
Params:  {'RSI_BULL': 15, 'RSI_BULL_TF': 5, 'RSI_BULL_HIGH': 67.66238588592422, 'RSI_BULL_LOW': 57.20343919552216, 'RSI_BEAR': 14, 'RSI_BEAR_TF': 9, 'RSI_BEAR_HIGH': 31.293003736740065, 'RSI_BEAR_LOW': 17.304646843870938, 'SMA_FAST': 138, 'SML_SLOW': 67, 'SMA_TF': 13, 'ADX': 20, 'ADX_TF': 13, 'ADX_HIGH': 89.28179742476595, 'ADX_LOW': 31.402923028664258, '

KeyboardInterrupt: 